I want to convey two things in this notebook.
## 1. Don't have to be hesitant about using Loop.
They say "avoid loops!'.
But I think It's not bad idea to use loops for this competition.
Because:
* We have to use small batch inference using Time-series API.
* Loops have very small overhead for each batch.
* Loops are more flexible.
* Even loops are not so slow. 3 features are extracted within 10 minits for 100M train data, as you can see blow.

## 2. Future information should not be used.
Time-series API doesn't allow us to use information from the future.
So we should not use it, especially user statistics from future make things very bad.

In [1]:
import pickle
import pandas as pd
import numpy as np
import gc
from sklearn.metrics import roc_auc_score
from collections import defaultdict, deque
from tqdm.notebook import tqdm
import lightgbm as lgb

## setting
CV files are generated by [this notebook](https://www.kaggle.com/its7171/cv-strategy)

In [2]:
train_pickle = '../input/riiid-cross-validation-files/cv1_train.pickle'
valid_pickle = '../input/riiid-cross-validation-files/cv1_valid.pickle'
question_file = '../input/features/question3.csv'
debug = False
validaten_flg = False

## feature engineering

In [3]:
import pickle

loaded_pickle = open("../input/for-transformer/train_current_part.pkl", "rb")
train_current_part = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/train_current_question.pkl", "rb")
train_current_question = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/train_current_tag.pkl", "rb")
train_current_tag = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/train_other_feats.pkl", "rb")
train_other_feats = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/train_past_part_answ.pkl", "rb")
train_past_part_answ = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/train_past_quest_answ.pkl", "rb")
train_past_quest_answ = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/train_past_tag_answ.pkl", "rb")
train_past_tag_answ = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/train_y.pkl", "rb")
train_y = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/train_prior_exp.pkl", "rb")
train_prior_exp = pickle.load(loaded_pickle)



loaded_pickle = open("../input/for-transformer/valid_current_part.pkl", "rb")
valid_current_part = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/valid_current_question.pkl", "rb")
valid_current_question = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/valid_current_tag.pkl", "rb")
valid_current_tag = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/valid_other_feats.pkl", "rb")
valid_other_feats = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/valid_past_part_answ.pkl", "rb")
valid_past_part_answ = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/valid_past_quest_answ.pkl", "rb")
valid_past_quest_answ = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/valid_past_tag_answ.pkl", "rb")
valid_past_tag_answ = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/valid_y.pkl", "rb")
valid_y = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/valid_prior_exp.pkl", "rb")
valid_prior_exp = pickle.load(loaded_pickle)

In [4]:
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM, Masking, Embedding, Concatenate, Input, Reshape,Flatten, AveragePooling1D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.metrics import AUC
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Lambda
#from tensorflow.keras.layers import merge
from tensorflow.keras.layers import multiply, Reshape
import pandas as pd
import numpy as np
import gc
from sklearn.metrics import roc_auc_score
from collections import defaultdict
from tqdm import tqdm
from tqdm import trange
from tensorflow.keras.utils import Sequence

## modeling

In [5]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates


def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)




def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

  # add extra dimensions to add the padding
  # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)




def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)




def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead) 
    but it must be broadcastable for addition.

    Args:
      q: query shape == (..., seq_len_q, depth)
      k: key shape == (..., seq_len_k, depth)
      v: value shape == (..., seq_len_v, depth_v)
      mask: Float tensor with shape broadcastable 
            to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
      output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights




class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention, 
                                       (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights





def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
        tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])


    
    
    
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.2):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, e, training, mask):

        attn_output, _ = self.mha(x, x, e, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(e + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2



class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff,
                   maximum_position_encoding, rate=0.2):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        #self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, 
                                                self.d_model)


        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                           for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, e, training, mask):

        seq_len = tf.shape(x)[1]

        # adding embedding and position encoding.
        #x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)
    
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, e, training, mask)

        return x  # (batch_size, input_seq_len, d_model)


class Transformer(tf.keras.Model):
    def __init__(self, num_layers, en_d_model, en_num_heads, dff, pe_input, rate=0.2):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, en_d_model, en_num_heads, dff, 
                           pe_input, rate)


        self.second_final_layer = tf.keras.layers.Dense(dff)
        self.final_layer = Dense(1,activation = 'sigmoid')
    
    def call(self, inp1, inp2, training, mask):

        enc_output = self.encoder(inp1, inp2, training, mask)  # (batch_size, inp_seq_len, d_model)
            
        second_final_output = self.second_final_layer(enc_output)  # (batch_size, tar_seq_len, question_answer_pair_size)
        final_output = self.final_layer(second_final_output)
        return final_output



In [6]:
train_other_feats.shape

(622657, 60, 2)

In [7]:
train_other_feats1 = np.reshape(train_other_feats[:,:,0],(-1,60,1))
train_other_feats2 = np.reshape(train_other_feats[:,:,1],(-1,60,1))

valid_other_feats1 = np.reshape(valid_other_feats[:,:,0],(-1,60,1))
valid_other_feats2 = np.reshape(valid_other_feats[:,:,1],(-1,60,1))

In [8]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [9]:
num_layers = 4
d_model = 128
num_heads = 4
dff = 512
question_answer_pair_size = 27048
tag_answer_pair_size = 3042
part_answer_pair_size = 16
n_question = 13524
n_tag = 1521
n_part = 8
n_prev_exp = 4

pe_input = 60



def build(num_layers, d_model, num_heads, dff, question_answer_pair_size, tag_answer_pair_size, part_answer_pair_size
          , n_question, n_tag, n_part, n_prev_exp, pe_input):
    masking_func = lambda inputs, previous_mask: previous_mask
    en_input1 = Input(batch_shape = (None, None), name = 'question_answer_pair')
    en_input1_embed = Embedding(question_answer_pair_size, d_model)(en_input1)
    en_input2 = Input(batch_shape = (None, None), name = 'tag_answer_pair')
    en_input2_embed = Embedding(tag_answer_pair_size, d_model)(en_input2)
    en_input3 = Input(batch_shape = (None, None), name = 'part_answer_pair')
    en_input3_embed = Embedding(part_answer_pair_size, d_model)(en_input3)
    en_input7 = Input(batch_shape = (None, None, 1), name = 'other_feature1')
    en_input7_masked = (Masking(mask_value= 0, input_shape = (None, None, 1)))(en_input7)
    en_input7_embed = Dense(d_model, input_shape = (None, None, 1))(en_input7_masked)
    en_input8 = Input(batch_shape = (None, None, 1), name = 'other_feature2')
    en_input8_masked = (Masking(mask_value= 0, input_shape = (None, None, 1)))(en_input8)
    en_input8_embed = Dense(d_model, input_shape = (None, None, 1))(en_input8_masked)
    en_input9 = Input(batch_shape = (None, None), name = 'prev_q_exp')
    en_input9_embed = Embedding(n_prev_exp, d_model)(en_input9)
    
    
    en_input_embed_sum = tf.math.add_n([en_input1_embed, en_input2_embed, en_input3_embed, en_input7_embed, en_input8_embed, en_input9_embed])
    
    #en_input1_embed = K.sum(en_input1_embed, axis = -2)
    en_input4 = Input(batch_shape = (None, None), name = 'current_question')
    en_input4_embed = Embedding(n_question, d_model)(en_input4)
    en_input5 = Input(batch_shape = (None, None), name = 'current_tag')
    en_input5_embed = Embedding(n_tag, d_model)(en_input5)
    en_input6 = Input(batch_shape = (None, None), name = 'current_part')
    en_input6_embed = Embedding(n_part, d_model)(en_input6)
    en_input_embed_sum2 = tf.math.add_n([en_input4_embed, en_input5_embed, en_input6_embed])
    
    
    #en_input2_embed = K.sum(en_input2_embed, axis = -2)

    
    
    look_ahead_mask = create_look_ahead_mask(tf.shape(en_input_embed_sum)[1])
    padding_mask = create_padding_mask(en_input1)
    combined_mask = tf.maximum(look_ahead_mask, padding_mask)
    
    
    transformer = Transformer(num_layers, d_model, num_heads, dff, pe_input)
    
    final_output = transformer(en_input_embed_sum, en_input_embed_sum2, True, combined_mask)

    
    model = Model(inputs=[en_input1, en_input2, en_input3, en_input4, en_input5, en_input6, en_input7, en_input8, en_input9], outputs=final_output)
    #model.compile( optimizer = 'adam',
    #                loss = 'binary_crossentropy',
    #                metrics=['accuracy',AUC()])
    
    return model

#my_model = build(num_layers, d_model, num_heads, dff, question_answer_pair_size, tag_answer_pair_size, part_answer_pair_size
#          , n_question, n_tag, n_part, n_prev_exp, pe_input)



In [10]:
with tpu_strategy.scope():
    my_model = build(num_layers, d_model, num_heads, dff, question_answer_pair_size, tag_answer_pair_size, part_answer_pair_size
                      , n_question, n_tag, n_part, n_prev_exp, pe_input)
    my_model.compile( optimizer = 'adam',
                        loss = 'binary_crossentropy',
                        metrics=['accuracy',AUC()])

In [11]:
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max', restore_best_weights=True) 

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='Transformer_model_feature_extraction.h5', monitor='val_auc', mode='max', save_best_only=True, save_weights_only=True, verbose=1)

my_model.fit([train_past_quest_answ, train_past_tag_answ, train_past_part_answ, 
                               train_current_question, train_current_tag, train_current_part,
                               train_other_feats1,train_other_feats2, train_prior_exp],train_y, 
             validation_data=([valid_past_quest_answ, valid_past_tag_answ, valid_past_part_answ,
                               valid_current_question, valid_current_tag, valid_current_part,
                               valid_other_feats1, valid_other_feats2, valid_prior_exp],valid_y), batch_size = 200,
             epochs = 100, verbose = 1, callbacks = [earlyStopping, cp_callback])

Epoch 1/100
3114/3114 [==============================] - ETA: 0s - auc: 0.7690 - accuracy: 0.7223 - loss: 0.5313
Epoch 00001: val_auc improved from -inf to 0.82537, saving model to Transformer_model_feature_extraction.h5
3114/3114 [==============================] - 123s 39ms/step - auc: 0.7690 - accuracy: 0.7223 - loss: 0.5313 - val_auc: 0.8254 - val_accuracy: 0.7457 - val_loss: 0.4858
Epoch 2/100
3113/3114 [============================>.] - ETA: 0s - auc: 0.7961 - accuracy: 0.7379 - loss: 0.5084
Epoch 00002: val_auc improved from 0.82537 to 0.83811, saving model to Transformer_model_feature_extraction.h5
3114/3114 [==============================] - 109s 35ms/step - auc: 0.7961 - accuracy: 0.7379 - loss: 0.5084 - val_auc: 0.8381 - val_accuracy: 0.7601 - val_loss: 0.4665
Epoch 3/100
3114/3114 [==============================] - ETA: 0s - auc: 0.8039 - accuracy: 0.7427 - loss: 0.5015
Epoch 00003: val_auc improved from 0.83811 to 0.83979, saving model to Transformer_model_feature_extractio

In [12]:
#my_model.save_weights('Transformer_model_feature_extraction.h5')

Have a fun with loops! :)

In [13]:
#my_model.load_weights('../input/transformer-model/Transformer_model_feature_extraction.h5')

In [14]:
y_hat = my_model.predict([valid_past_quest_answ, valid_past_tag_answ, valid_past_part_answ,
                               valid_current_question, valid_current_tag, valid_current_part,
                               valid_other_feats1, valid_other_feats2, valid_prior_exp], batch_size = 200)

In [15]:
y_hat = y_hat[:,-1,0]
y = valid_y[:,-1,0]
print(roc_auc_score(y,y_hat))

0.7656748294398391
